In [ ]:
import cv2
import numpy as np
import math
from collections import deque, Counter

cap = cv2.VideoCapture(0)
gesture_buffer = deque(maxlen=10)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.rectangle(frame, (100, 100), (400, 400), (0, 255, 0), 0)
    roi = frame[100:400, 100:400]

    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 30, 60])
    upper_skin = np.array([20, 150, 255])
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.dilate(mask, kernel, iterations=4)

    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        cnt = max(contours, key=lambda x: cv2.contourArea(x))

        hull_points = cv2.convexHull(cnt)
        hull_indices = cv2.convexHull(cnt, returnPoints=False)

        if len(hull_indices) > 3 and cnt.shape[0] > 5:
            defects = cv2.convexityDefects(cnt, hull_indices)

            count_defects = 0
            if defects is not None:
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i, 0]
                    start = tuple(cnt[s][0])
                    end = tuple(cnt[e][0])
                    far = tuple(cnt[f][0])

                    a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                    b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                    c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                    angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 57

                    if angle <= 90 and d > 10000:
                        count_defects += 1

            area_cnt = cv2.contourArea(cnt)
            area_hull = cv2.contourArea(hull_points)
            ratio = area_cnt / area_hull if area_hull != 0 else 0

            left = tuple(hull_points[hull_points[:, :, 0].argmin()][0])
            right = tuple(hull_points[hull_points[:, :, 0].argmax()][0])
            top = tuple(hull_points[hull_points[:, :, 1].argmin()][0])
            bottom = tuple(hull_points[hull_points[:, :, 1].argmax()][0])

            center_x = (left[0] + right[0]) // 2
            center_y = (top[1] + bottom[1]) // 2

            dist_top = center_y - top[1]
            dist_bottom = bottom[1] - center_y
            dist_left = center_x - left[0]
            dist_right = right[0] - center_x

            gesture = "unrecognized"
            if count_defects == 0:
                if ratio > 0.85:
                    gesture = "fist"
                else:
                    max_dist_dir = max(dist_top, dist_bottom, dist_left, dist_right)
                    if max_dist_dir == dist_top:
                        gesture = "thumbs up"
                    elif max_dist_dir == dist_bottom:
                        gesture = "thumbs down"
            elif count_defects == 1:
                gesture = "peace"
            elif count_defects == 4:
                gesture = "hello"

            gesture_buffer.append(gesture)
            count = Counter(gesture_buffer)
            display_gesture = count.most_common(1)[0][0] if count else "unrecognized"

            cv2.putText(frame, display_gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)

    cv2.imshow("Gesture", frame)
    cv2.imshow("Mask", mask)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()